In [ ]:
1. Importations et chargement du dataset

In [26]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Charger le dataset Malaria
dataset, info = tfds.load('malaria', as_supervised=True, with_info=True)

# Informations sur le dataset
print('Dataset name:', info.name)
print('Dataset description:', info.description)
print('Dataset version:', info.version)
print('Dataset features:', info.features)
print('Dataset splits:', info.splits)
print('Number of train examples:', info.splits['train'].num_examples)
print('Number of classes:', info.features['label'].num_classes)

# Extraction des données
full_data = dataset['train']
total_examples = info.splits['train'].num_examples

# Ratios pour les splits
train_split, val_split, test_split = 0.8, 0.1, 0.1
train_size = int(total_examples * train_split)
val_size = int(total_examples * val_split)
test_size = total_examples - train_size - val_size

# Mélange et séparation des données
shuffled_data = full_data.shuffle(total_examples, seed=42)
train_data = shuffled_data.take(train_size)
remaining_data = shuffled_data.skip(train_size)
val_data = remaining_data.take(val_size)
test_data = remaining_data.skip(val_size)


Dataset name: malaria
Dataset description: The Malaria dataset contains a total of 27,558 cell images with equal instances
of parasitized and uninfected cells from the thin blood smear slide images of
segmented cells.
Dataset version: 1.0.0
Dataset features: FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
})
Dataset splits: {'train': <SplitInfo num_examples=27558, num_shards=4>}
Number of train examples: 27558
Number of classes: 2


2. Affichage des images de l'ensemble de données

In [27]:
class_names = ['Parasitized', 'Uninfected']

def display_images(dataset, num_images=9):
    plt.figure(figsize=(10, 10))

    # Initialisation du compteur d'images
    images_shown = 0

    for batch_images, batch_labels in dataset:
        batch_images = batch_images.numpy()  # Convertir les images en NumPy
        batch_labels = batch_labels.numpy()  # Convertir les labels en NumPy

        for image, label in zip(batch_images, batch_labels):
            if images_shown >= num_images:
                break

            # Affichage de l'image
            plt.subplot(3, 3, images_shown + 1)
            plt.imshow(image)  # L'image est déjà normalisée
            plt.title(class_names[label])  # Nom de la classe
            plt.axis('off')

            images_shown += 1

        if images_shown >= num_images:
            break

    plt.tight_layout()
    plt.show()

# Visualiser les images
print("Images Train Data")
display_images(train_data, num_images=9)

print("Images Test Data")
display_images(test_data, num_images=9)

print("Images Validation Data")
display_images(val_data, num_images=9)


Images Train Data


TypeError: 'numpy.int64' object is not iterable

<Figure size 1000x1000 with 0 Axes>

3. Visualisation de la distribution des classes

In [ ]:
def plot_class_distribution(data, dataset_name, chart_type='bar'):
    # Débatcher et extraire les labels
    labels = np.array([label.numpy() for _, label in data.unbatch()])

    # Compter les occurrences des classes
    unique, counts = np.unique(labels, return_counts=True)

    # Visualisation selon le type de graphique
    if chart_type == 'bar':
        plt.figure(figsize=(6, 4))
        plt.bar(unique, counts, tick_label=class_names, color=['skyblue', 'salmon'])
        plt.xlabel('Classes')
        plt.ylabel('Nombre d\'exemples')
        plt.title(f'Distribution des classes dans {dataset_name}')
        plt.show()
    elif chart_type == 'pie':
        plt.figure(figsize=(6, 6))
        plt.pie(counts, labels=class_names, autopct='%1.1f%%', colors=['skyblue', 'salmon'], startangle=90)
        plt.title(f'Distribution des classes dans {dataset_name}')
        plt.show()

# Visualisation en barres
print("Distribution des classes (bar chart):")
plot_class_distribution(train_data, 'Train Data', chart_type='bar')
plot_class_distribution(val_data, 'Validation Data', chart_type='bar')
plot_class_distribution(test_data, 'Test Data', chart_type='bar')

# Visualisation en camembert
print("Distribution des classes (pie chart):")
plot_class_distribution(train_data, 'Train Data', chart_type='pie')
plot_class_distribution(val_data, 'Validation Data', chart_type='pie')
plot_class_distribution(test_data, 'Test Data', chart_type='pie')

4. Prétraitement des images et préparation des données

In [ ]:
image_size = 224

# Fonction de prétraitement des images
def preprocess_image(image, label):
    image = tf.image.resize(image, [image_size, image_size])  # Redimensionner à 224x224
    image = image / 255.0  # Normalisation
    label = tf.cast(label, tf.int32)  # Conversion des labels en int
    return image, label

# Chargement et prétraitement des données
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.map(preprocess_image, num_parallel_calls=AUTOTUNE)
val_data = val_data.map(preprocess_image, num_parallel_calls=AUTOTUNE)
test_data = test_data.map(preprocess_image, num_parallel_calls=AUTOTUNE)

train_data = train_data.batch(32).prefetch(AUTOTUNE)
val_data = val_data.batch(32).prefetch(AUTOTUNE)
test_data = test_data.batch(32).prefetch(AUTOTUNE)

5. Définition du modèle MobileNetV2 et compilation

In [ ]:
# Définir MobileNetV2 avec la taille d'entrée de 224x224
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Gel des poids du modèle de base pour éviter de les entraîner
base_model.trainable = False

# Création du modèle complet
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Sigmoid pour 2 classes
])

# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Affichage du résumé du modèle pour vérifier les couches
model.summary()

6. Définition des callbacks et planification du taux d'apprentissage

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

# Fonction de planification du taux d'apprentissage (learning rate)
def lr_schedule(epoch):
    if epoch < 10:
        return 0.001  # Taux d'apprentissage pour les 10 premières époques
    else:
        return 0.0001  # Taux d'apprentissage après 10 époques

# Callback pour ajuster dynamiquement le learning rate
lr_scheduler = LearningRateScheduler(lr_schedule)

# Callback pour l'arrêt précoce, surveille la 'val_loss' et arrête l'entraînement si la performance ne s'améliore pas
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=3,  # Arrêt après 3 époques sans amélioration
                               restore_best_weights=True,  # Restaure les meilleurs poids trouvés
                               verbose=2)

# Callback pour réduire dynamiquement le learning rate en cas de stagnation de la validation loss
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,  # Divise le learning rate par 2
                              patience=2,  # Attend 2 époques sans amélioration avant de réduire le LR
                              verbose=2)

# Liste de callbacks pour les intégrer dans l'entraînement
callbacks = [lr_scheduler, early_stopping, reduce_lr]

7. Entraînement du modèle

In [ ]:
# Paramètres d'entraînement
steps_per_epoch = train_size // 32  # Calcul du nombre de steps pour chaque époque
validation_steps = val_size // 32  # Nombre de steps pour la validation

# Entraîner le modèle avec des callbacks
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    steps_per_epoch=steps_per_epoch,  # Nombre de steps pour chaque époque
    validation_steps=validation_steps,  # Nombre de steps pour la validation
    callbacks=callbacks,  # Ajouter les callbacks ici
    verbose=1  # Affichage de la barre de progression
)